In [ ]:
from pathlib import Path

from dateutil.parser import parse as parse_date

In [ ]:
# === Define project paths ===
project_name = "aoi_3_bologna"

# Directory where HyP3 products (SLCs/interferograms) are stored
data_dir = Path("/mnt/data") / project_name
hyp_int_dir = data_dir / "hyp_int"  # For interferograms only

# Working directory for MintPy, config files, etc.
work_dir = Path.home() / "work" / "aoi_bologna" / "aoi_3_1"

# Create necessary folders if they don't exist
for d in [data_dir, hyp_int_dir, work_dir]:
    d.mkdir(parents=True, exist_ok=True)

print("✔️ Project setup complete:")
print(f"  - Data directory:      {data_dir}")
print(f"  - HyP3 interferograms: {hyp_int_dir}")
print(f"  - Work directory:      {work_dir}")


In [ ]:
# --- Define search configuration ---
stack_start = parse_date("2016-12-31T23:00:00Z")
stack_end = parse_date("2025-06-30T21:59:59Z")
max_temporal_baseline = 24  # days, used later for SBAS pair filtering (optional)

In [ ]:
import asf_search as asf
from pprint import pprint


In [ ]:

# Define ASF search parameters
opts = asf.ASFSearchOptions(**{
    "maxResults": 5000,
    "bbox": [11.2, 44.4, 11.5, 44.6],  # Bologna AOI (EPSG:32633)
    "beamSwath": ["IW"],
    "flightDirection": "DESCENDING",
    "polarization": ["VV+VH", "VV"],
    "processingLevel": ["SLC"],
    "start": stack_start.isoformat(),
    "end": stack_end.isoformat(),
    "dataset": ["SENTINEL-1"]
})

# Search SLC scenes from ASF
search_results = asf.search(opts=opts)
print(f"🔍 Found {len(search_results)} scenes.")

# Create baseline stack (SBAS-compatible) from the most recent scene
baseline_results = asf.baseline_search.stack_from_product(search_results[-1])
print(f"Generated {len(baseline_results)} SBAS pairs.")

# Preview first few SBAS pairs
pprint(baseline_results[:3])


In [ ]:
import pandas as pd


# Dynamically get all properties
columns = list(baseline_results[0].properties.keys()) + ['geometry']
data = [list(scene.properties.values()) + [scene.geometry] for scene in baseline_results]

# Create DataFrame
stack = pd.DataFrame(data, columns=columns)

# Show available columns to inspect
print("Available DataFrame columns:")
print(stack.columns.tolist())

# Convert to datetime
stack['startTime'] = stack['startTime'].apply(parse_date)

# Filter by date range
stack = stack.loc[(stack_start <= stack.startTime) & (stack.startTime <= stack_end)]

# Show preview using safe subset
preview_cols = [col for col in ['sceneName', 'startTime', 'stopTime'] if col in stack.columns]
print(f"Filtered SBAS stack: {len(stack)} pairs")
stack[preview_cols].head()


In [ ]:
sbas_pairs = set()

for reference, rt in stack.loc[::-1, ['sceneName', 'temporalBaseline']].itertuples(index=False):
    secondaries = stack.loc[
        (stack.sceneName != reference)
        & (stack.temporalBaseline - rt <= max_temporal_baseline)
        & (stack.temporalBaseline - rt > 0)
    ]
    for secondary in secondaries.sceneName:
        sbas_pairs.add((reference, secondary))

In [ ]:
import json

# Save stack metadata (SLCs with all properties)
stack_csv_path = work_dir / "stack_scenes.csv"
stack_json_path = work_dir / "stack_scenes.json"

stack.to_csv(stack_csv_path, index=False)
stack.to_json(stack_json_path, orient="records", indent=2)

# Save SBAS pairs
sbas_pairs_list = [{"reference": ref, "secondary": sec} for ref, sec in sbas_pairs]
sbas_csv_path = work_dir / "sbas_pairs.csv"
sbas_json_path = work_dir / "sbas_pairs.json"

pd.DataFrame(sbas_pairs_list).to_csv(sbas_csv_path, index=False)
with open(sbas_json_path, "w") as f:
    json.dump(sbas_pairs_list, f, indent=2)

print(f"Saved stack and SBAS pairs to: {work_dir}")


## 2. Request On Demand InSAR products from ASF HyP3

# Use your [NASA Earthdata login](https://urs.earthdata.nasa.gov/) to connect to [ASF HyP3](https://hyp3-docs.asf.alaska.edu/).

In [ ]:
import hyp3_sdk as sdk
hyp3 = sdk.HyP3()  # Automatically uses ~/.netrc

# Initialize HyP3 with interactive login if needed
# hyp3 = sdk.HyP3(prompt=True)


In [ ]:
from tqdm import tqdm
import time
import hyp3_sdk as sdk

# Initialize
hyp3 = sdk.HyP3()
jobs = sdk.Batch()

# Submit with progress bar
for reference, secondary in tqdm(sbas_pairs, desc="Submitting InSAR jobs"):
    jobs += hyp3.submit_insar_job(
        reference, secondary,
        name=project_name,
        include_dem=True,
        include_look_vectors=True,
        include_wrapped_phase=True,
        include_los_displacement=True,
        include_displacement_maps=True
    )
    time.sleep(0.2)  # Avoid rate limiting



In [ ]:

jobs = hyp3.find_jobs()  # This lists all your jobs


In [ ]:
# Save the jobs object to a file for later use
import pickle

# Set the path to your target file inside work_dir
job_pickle_path = work_dir / 'hyp3_jobs.pkl'

# jobs is your HyP3JobList object from hyp3.find_jobs()
with open(job_pickle_path, 'wb') as f:
    pickle.dump(jobs, f)


In [ ]:
# Fetch all your jobs

# Show keys of the first job's `job_parameters` safely
first_job = jobs[0]

first_job_dict = first_job.to_dict()

print("Top-level job keys:")
print(first_job_dict.keys())

print("\nJob parameter keys:")
print(first_job_dict['job_parameters'].keys())



In [ ]:
all_jobs = hyp3.find_jobs(name="aoi_3_bologna")
print(f"Found {len(all_jobs)} jobs")
statuses = [job.status_code for job in all_jobs]
from collections import Counter
print(Counter(statuses))


In [ ]:
my_project_jobs = [job for job in all_jobs 
                   if job.job_type == "INSAR_GAMMA" and job.name == project_name]


In [ ]:
Counter(job.status_code for job in my_project_jobs)


In [ ]:
print(f"{first_job.credit_cost}")

total_credits = sum(j.credit_cost for j in my_project_jobs)
print(f"Total credit cost for 'aoi_3_bologna': {total_credits}")

In [ ]:
for job in my_project_jobs:
    print(f"{job.job_id} — {job.job_type} — {job.status_code} — {job.name} — {job.request_time}")


In [ ]:
for job in my_project_jobs:
    granules = job.job_parameters['granules']
    print(f"{granules[0]} <---> {granules[1]}")


In [ ]:
import pprint
pprint.pprint(my_project_jobs[0].job_parameters)


In [ ]:
from tqdm import tqdm

# Only keep succeeded jobs
succeeded_jobs = [job for job in my_project_jobs if job.status_code == 'SUCCEEDED']


In [ ]:

downloaded_files = []

for job in tqdm(succeeded_jobs, desc="Downloading SUCCEEDED HyP3 jobs"):
    downloaded = job.download_files(data_dir)
    downloaded_files.extend(downloaded)


In [ ]:
import zipfile
import os
import pandas as pd
from datetime import datetime

records = []

for job in succeeded_jobs:
    # Get ZIP file name from job
    zip_path = os.path.join(data_dir, job.files[0]['filename'])
    
    if not os.path.exists(zip_path):
        continue  # skip if not downloaded
    
    zip_size = os.path.getsize(zip_path) / (1024**2)  # size in MB
    
    # Extract dates from granules
    granules = job.job_parameters['granules']
    reference = granules[0]
    secondary = granules[1]
    ref_date = reference.split('_')[5][:8]
    sec_date = secondary.split('_')[5][:8]

    # Compose interferogram basename
    intf_name = f"{ref_date}_{sec_date}"
    
    records.append({
        "job_id": job.job_id,
        "name": job.name,
        "status": job.status_code,
        "request_time": job.request_time,
        "credit_cost": job.credit_cost,
        "granule_1": reference,
        "granule_2": secondary,
        "ref_date": ref_date,
        "sec_date": sec_date,
        "job_type": job.job_type,
        "looks": job.job_parameters.get("looks"),
        "include_dem": job.job_parameters.get("include_dem"),
        "include_look_vectors": job.job_parameters.get("include_look_vectors"),
        "include_displacement_maps": job.job_parameters.get("include_displacement_maps"),
        "include_los_displacement": job.job_parameters.get("include_los_displacement"),
        "include_wrapped_phase": job.job_parameters.get("include_wrapped_phase"),
        "apply_water_mask": job.job_parameters.get("apply_water_mask"),
        "phase_filter_parameter": job.job_parameters.get("phase_filter_parameter"),
        "zip_filename": os.path.basename(zip_path),
        "zip_file_size_MB": round(zip_size, 2),
        "interferogram_name": intf_name
    })

# Convert to DataFrame and save
df = pd.DataFrame.from_records(records)
df.to_csv(work_dir / "hyp3_interferogram_jobs.csv", index=False)

print(f"Saved metadata for {len(df)} jobs to hyp3_interferogram_jobs.csv")


In [ ]:
from tqdm import tqdm
import os

unzipped_dirs = []

for job in tqdm(succeeded_jobs, desc="Unzipping InSAR products"):
    zip_path = os.path.join(data_dir, job.files[0]['filename'])

    if not os.path.exists(zip_path):
        continue  # Skip if ZIP not downloaded

    unzip_folder = os.path.splitext(zip_path)[0]
    if os.path.exists(unzip_folder):
        print(f"Already unzipped: {unzip_folder}")
        unzipped_dirs.append(unzip_folder)
        continue

    try:
        extracted_path = sdk.util.extract_zipped_product(zip_path)
        unzipped_dirs.append(extracted_path)
    except Exception as e:
        print(f"Failed to unzip {zip_path}: {e}")


In [ ]:
# Remove zipped files - after checking the zipping pocess was complete 

import os
from pathlib import Path
from zipfile import ZipFile

from tqdm import tqdm
from hyp3_sdk.util import extract_zipped_product

# Assuming `data_dir` is already defined
zipped_files = sorted(Path(data_dir).glob("*.zip"))

print(f"Found {len(zipped_files)} .zip files in {data_dir}")

for zip_path in tqdm(zipped_files, desc="Processing ZIPs"):
    zip_path = Path(zip_path)
    # Expected unzipped folder name (same as zip without .zip)
    unzipped_folder = zip_path.with_suffix('')

    if unzipped_folder.exists():
        # Already unzipped, safe to delete zip
        zip_path.unlink()
    else:
        # Not yet unzipped → unzip then delete zip
        try:
            extract_zipped_product(zip_path)
            zip_path.unlink()
        except Exception as e:
            print(f"Failed to extract {zip_path.name}: {e}")


In [ ]:
# Calculate ZIP file sizes and add to CSV

import os
import pandas as pd
from pathlib import Path

# === Paths ===
csv_path = work_dir / "hyp3_interferogram_jobs.csv"  # already defined
data_dir = Path("/mnt/data/aoi_3_bologna")       # already defined

# === Load CSV ===
df = pd.read_csv(csv_path)

# === Initialize size column ===
zip_sizes_mb = []
missing_files = []

for fname in df['zip_filename']:  # adjust column name if needed
    zip_path = data_dir / fname
    if zip_path.exists():
        size_mb = zip_path.stat().st_size / (1024 ** 2)
        zip_sizes_mb.append(size_mb)
    else:
        zip_sizes_mb.append(None)
        missing_files.append(fname)

# === Add column and save ===
df['zip_size_MB'] = zip_sizes_mb
df.to_csv(csv_path, index=False)

# === Summary ===
total_size_mb = sum(size for size in zip_sizes_mb if size is not None)
print(f"✅ Total ZIP size: {total_size_mb:.2f} MB")
if missing_files:
    print(f"⚠️ Missing ZIP files: {len(missing_files)}")
    for f in missing_files:
        print(" -", f)


In [ ]:
# Inspecting a specific job object
job = succeeded_jobs[0]  # or any index

# Check object type
print("Type of job:", type(job))

# See all top-level attributes
print("\nTop-level attributes:")
print(job.__dict__.keys())

# See the full object using pprint
from pprint import pprint
pprint(job.__dict__)


In [ ]:
# Download GPS time series for a specific station
import requests
from pathlib import Path
from io import StringIO
import pandas as pd
from tqdm import tqdm

# === Setup ===
station = "MEDI"
gps_dir = data_dir / "gps"
gps_dir.mkdir(exist_ok=True)
gps_path = gps_dir / f"{station}.EU.tenv3"


In [ ]:

# === Download ===
url = f"https://geodesy.unr.edu/gps_timeseries/tenv3/plates/EU/{station}.EU.tenv3"

response = requests.get(url)
response.raise_for_status()
with open(gps_path, 'w') as f:
    f.write(response.text)

print(f"✅ Downloaded {station} time series to {gps_path}")


In [ ]:
# === Read and process GPS data ===
import pandas as pd

# Define correct column names based on your structure
col_names = [
    "site", "YYMMMDD", "decimal_year", "MJD", "week", "doy",
    "reflon", "e0", "east", "n0", "north", "u0", "up", "ant",
    "sig_e", "sig_n", "sig_u", "corr_en", "corr_eu", "corr_nu",
    "lat", "lon", "height"
]

# Read fixed-width file with manual column names
df_gps = pd.read_csv(gps_path, delim_whitespace=True, comment="#", names=col_names)

# Convert to numeric
df_gps['decimal_year'] = pd.to_numeric(df_gps['decimal_year'], errors='coerce')
df_gps['up'] = pd.to_numeric(df_gps['up'], errors='coerce')
df_gps['sig_u'] = pd.to_numeric(df_gps['sig_u'], errors='coerce')

# Drop rows with missing time or up values
df_gps = df_gps.dropna(subset=['decimal_year', 'up', 'sig_u'])


In [ ]:
lat = df_gps['lat'].iloc[0]
lon = df_gps['lon'].iloc[0]
print(f"📍 GPS station {station} at lat={lat}, lon={lon}")



In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.errorbar(df_gps['decimal_year'], df_gps['up'], yerr=df_gps['sig_u'],
             fmt='o', markersize=3, capsize=2, label='Up displacement', ecolor='gray')
plt.xlabel('Decimal Year')
plt.ylabel('Displacement (mm)')
plt.title('Vertical Displacement (Up) - GPS Station MEDI')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()



In [ ]:
from pathlib import Path
import cdsapi

c = cdsapi.Client()

for year in range(2016, 2026):
    outfile = data_dir / f"era5_tcwv_{year}.nc"
    if outfile.exists():
        print(f"{outfile.name} already downloaded.")
        continue

    c.retrieve(
        'reanalysis-era5-single-levels',
        {
            'product_type': 'reanalysis',
            'variable': ['total_column_water_vapour'],
            'year': str(year),
            'month': [f"{m:02d}" for m in range(1, 13)],
            'day': [f"{d:02d}" for d in range(1, 32)],
            'time': ['12:00'],
            'format': 'netcdf',
            'area': [44.6, 11.2, 44.4, 11.5],  # [north, west, south, east]
        },
        str(outfile)
    )


In [ ]:
import sys
sys.path.append('/home/ubuntu/tools/MintPy/src')

from mintpy.cli import prep_hyp3
from mintpy.utils import readfile, writefile
from pathlib import Path
import os


In [ ]:
print (f"Data directory: {data_dir}")

In [ ]:
from mintpy.utils import writefile
print(dir(writefile))  # check if 'write_roipac_rsc' is listed


In [ ]:
from mintpy import prep_hyp3
from mintpy.utils import readfile, writefile
from pathlib import Path


# Define file patterns to include
patterns = [
    "*unw_phase*.tif",
    "*corr*.tif",
    "*dem*.tif",
    "*lv_theta*.tif"
]

# Collect all matching files
all_tifs = []
for pat in patterns:
    all_tifs.extend(data_dir.rglob(pat))

# Loop through and write .rsc for each
for tif_file in sorted(all_tifs):
    try:
        print(f"Processing: {tif_file.name}")
        meta = readfile.read_gdal_vrt(str(tif_file))
        meta = prep_hyp3.add_hyp3_metadata(str(tif_file), meta)
        writefile.write_roipac_rsc(meta, str(tif_file) + ".rsc")
    except Exception as e:
        print(f"❌ Failed on {tif_file.name}: {e}")


In [ ]:
import subprocess
from pathlib import Path

mintpy_input_dir = Path("/home/ubuntu/work/aoi_bologna/aoi_3_1/mintpy_inputs")
mintpy_input_dir.mkdir(exist_ok=True)

cmd = f"""
prep_hyp3.py \
--dir "{data_dir}" \
--outdir "{mintpy_input_dir}" \
--file "**/*unw_phase_clipped.tif" \
--coherence "**/*corr_clipped.tif" \
--ex "**/*dem_clipped.tif" \
--theta "**/*lv_theta_clipped.tif"
"""

subprocess.run(cmd, check=True, shell=True)


In [ ]:
from pathlib import Path
from typing import List, Union
from osgeo import gdal

def get_common_overlap(file_list: List[Union[str, Path]]) -> List[float]:
    """Get the common overlap of GeoTIFF files with correct projWin bounds."""
    corners = [gdal.Info(str(file), format='json')['cornerCoordinates'] for file in file_list]

    ulx = min(corner['upperLeft'][0] for corner in corners)
    uly = max(corner['upperLeft'][1] for corner in corners)
    lrx = max(corner['lowerRight'][0] for corner in corners)
    lry = min(corner['lowerRight'][1] for corner in corners)

    

    return [ulx, uly, lrx, lry]


'''
def get_common_overlap(file_list: List[Union[str, Path]]) -> List[float]:
    """Get the common overlap of  a list of GeoTIFF files
    
    Arg:
        file_list: a list of GeoTIFF files
    
    Returns:
         [ulx, uly, lrx, lry], the upper-left x, upper-left y, lower-right x, and lower-right y
         corner coordinates of the common overlap
    """
    
    corners = [gdal.Info(str(dem), format='json')['cornerCoordinates'] for dem in file_list]

    ulx = max(corner['upperLeft'][0] for corner in corners)
    uly = min(corner['upperLeft'][1] for corner in corners)
    lrx = min(corner['lowerRight'][0] for corner in corners)
    lry = max(corner['lowerRight'][1] for corner in corners)
    return [ulx, uly, lrx, lry]
'''

In [ ]:
from osgeo import gdal
gdal.UseExceptions()
data_dir = Path(data_dir)
files = data_dir.glob('*/*_dem.tif')

overlap = get_common_overlap(files)

In [ ]:
print(overlap)

In [ ]:
##Modify later!

from osgeo import gdal

ds = gdal.Open("/path/to/any_dem_or_unw_phase_file.tif")
proj = ds.GetProjection()
print(gdal.Info(ds, format="json")['coordinateSystem'])

from shapely.geometry import box
from shapely.ops import transform
from pyproj import Transformer

# Define AOI in lon/lat (WGS84)
aoi_wgs84 = box(11.2, 44.4, 11.5, 44.6)

transformer = Transformer.from_crs("EPSG:4326", "EPSG:32633", always_xy=True)
aoi_utm33 = transform(transformer.transform, aoi_wgs84)

print("AOI in UTM 33N (EPSG:32633):", aoi_utm33.bounds)
(minx, miny, maxx, maxy) = aoi_utm33.bounds
# Equivalent to [ulx, uly, lrx, lry] for your AOI in meters
from shapely.geometry import box

overlap_box = box(overlap[0], overlap[3], overlap[2], overlap[1])  # minx, miny, maxx, maxy
print("Does overlap contain AOI?", overlap_box.contains(aoi_utm33))

In [ ]:
## Check this alternative later ... !
# Clip all GeoTIFF files to their common overlap
from pathlib import Path
from typing import List, Union
from osgeo import gdal
from tqdm import tqdm

def clip_hyp3_products_to_common_overlap(data_dir: Union[str, Path], overlap: List[float]) -> None:
    """Clip all GeoTIFF files to their common overlap

    Args:
        data_dir: directory containing the GeoTIFF files to clip
        overlap: [ulx, uly, lrx, lry] of the common overlap area
    Returns: None
    """

    data_dir = Path(data_dir)
    files_for_mintpy = ['_water_mask.tif', '_corr.tif', '_unw_phase.tif', '_dem.tif', '_lv_theta.tif', '_lv_phi.tif']

    for extension in files_for_mintpy:
        matching_files = list(data_dir.rglob(f'*{extension}'))
        for file in tqdm(matching_files, desc=f'Clipping {extension} files'):
            dst_file = file.parent / f'{file.stem}_clipped{file.suffix}'
            gdal.Translate(destName=str(dst_file), srcDS=str(file), projWin=overlap)


In [ ]:
from pathlib import Path
from typing import List, Union

def clip_hyp3_products_to_common_overlap(data_dir: Union[str, Path], overlap: List[float]) -> None:
    """Clip all GeoTIFF files to their common overlap
    
    Args:
        data_dir:
            directory containing the GeoTIFF files to clip
        overlap:
            a list of the upper-left x, upper-left y, lower-right-x, and lower-tight y
            corner coordinates of the common overlap
    Returns: None
    """

    
    files_for_mintpy = ['_water_mask.tif', '_corr.tif', '_unw_phase.tif', '_dem.tif', '_lv_theta.tif', '_lv_phi.tif']

    for extension in files_for_mintpy:

        for file in data_dir.rglob(f'*{extension}'):

            dst_file = file.parent / f'{file.stem}_clipped{file.suffix}'

            gdal.Translate(destName=str(dst_file), srcDS=str(file), projWin=overlap)

In [ ]:
from osgeo import gdal
gdal.UseExceptions()

clip_hyp3_products_to_common_overlap(data_dir, overlap)

In [ ]:
mintpy_config = work_dir / 'mintpy_config.txt'
mintpy_config.write_text(
f"""
##---------processor info:
mintpy.load.processor        = hyp3

##---------interferogram datasets:
mintpy.load.unwFile          = {data_dir}/*/*_unw_phase_clipped.tif
mintpy.load.corFile          = {data_dir}/*/*_corr_clipped.tif

##---------geometry datasets:
mintpy.load.demFile          = {data_dir}/*/*_dem_clipped.tif
mintpy.load.incAngleFile     = {data_dir}/*/*_lv_theta_clipped.tif
mintpy.load.azAngleFile      = {data_dir}/*/*_lv_phi_clipped.tif
mintpy.load.waterMaskFile    = {data_dir}/*/*_water_mask_clipped.tif

##---------tropospheric delay correction:
mintpy.troposphericDelay.method        = weatherModel
mintpy.weatherModel.weatherModel       = ERA5
mintpy.weatherModel.weatherDir         = {data_dir}
mintpy.weatherModel.weatherFile        = era5_tcwv_*.nc
mintpy.weatherModel.heightRef          = auto

##---------additional options:
mintpy.network.inBaseline              = yes
mintpy.network.maxTempBaseline         = 180
mintpy.network.maxBTempBaseline        = 1500

mintpy.unwrapMask.connCompMinPix       = 100
mintpy.unwrapMask.maskFile             = auto

mintpy.waterBody.mask                  = auto
mintpy.reference.pixelMethod           = lonlat
mintpy.reference.lalo                  = 44.5199558120, 11.6468120447  # MEDI GPS lat, lon (adjusted from -348° to +11.65°)
mintpy.reference.interpMethod          = linear
mintpy.tempCohMask.min                 = 0.7
mintpy.deramp                          = quadratic
""")


In [ ]:
!smallbaselineApp.py --dir {work_dir} {mintpy_config}